### Navegador

In [61]:
# Criar navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [62]:
import os
# Abrir link index
caminho = os.getcwd()
arquivo = caminho + r"\index.html"


In [63]:
import pandas as pd
#Importar base de dados
tabela = pd.read_excel("Processos.xlsx")

tabela["Status"] = tabela["Status"].astype(str)
display(tabela)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,nan
1,João,Lawyer Alon,EB3792,Rio de Janeiro,nan
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,nan
3,Carol,Amanda,PC5197,São Paulo,nan


In [64]:
from selenium.webdriver import ActionChains
import time

for linha in tabela.index:

    # Para cada tabela consultar
    navegador.get(arquivo)
    # Escolher um Estado
    botao = navegador.find_element(By.CLASS_NAME, 'dropdown-menu')
    ActionChains(navegador).move_to_element(botao).perform()

    cidade = tabela.loc[linha, "Cidade"]
    # Selecionando o rio
    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()
    
    aba_original = navegador.window_handles[0]
    indice = 1 + linha
    aba_nova = navegador.window_handles[indice]
    navegador.switch_to.window(aba_nova)
    # Preencher o forms com os dados de busca

 
    navegador.find_element(By.ID, 'nome').send_keys(tabela.loc[linha, "Nome"])
    navegador.find_element(By.ID, 'advogado').send_keys(tabela.loc[linha, "Advogado"])
    navegador.find_element(By.ID, 'numero').send_keys(tabela.loc[linha, "Processo"])
    #Clicar em Pesquisar
    navegador.find_element(By.CLASS_NAME, "registerbtn").click()
    # confirmar a pesquisa
    alerta = navegador.switch_to.alert
    alerta.accept()


    while True:
        try:
            alerta = navegador.switch_to.alert
            break
        except:
            time.sleep(1)
    texto_alerta = alerta.text

    if "Processo encontrado com sucesso" in texto_alerta:
        alerta.accept()
        tabela.loc[linha, "Status"] = "Encontrado"
    else:
        tabela.loc[linha, "Status"] = "Não Encontrado"
        alerta.accept()

    # Esperar o resultado da pesquisa e agir de acordo com o resultado

In [ ]:
display(tabela)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Encontrado
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Encontrado
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Encontrado
3,Carol,Amanda,PC5197,São Paulo,Encontrado
